In [1]:
import math
import nltk
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.probability import FreqDist

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
import spacy
nlp = spacy.load('en_core_web_sm')

stop_words_list = []
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#Metodi ausiliari

In [2]:
# Ritorna il POS TAG di una parola
def get_wordnet_pos(word):
  treebank_tag = [tag for (word, tag) in nltk.pos_tag(nltk.word_tokenize(word))][0]
  if treebank_tag.startswith('J'):
    return wn.ADJ
  elif treebank_tag.startswith('V'):
    return wn.VERB
  elif treebank_tag.startswith('N'):
    return wn.NOUN
  elif treebank_tag.startswith('R'):
    return wn.ADV
  else:
    return ''


# Ritorna una lista con le stop words
def get_stop_words():
  if len(stop_words_list) == 0:
    f = open("./stop_words_FULL.txt", "r")
    for x in f:
      stop_words_list.append(x)

  return stop_words_list + ['’']


# Da una frase ritorna una lista con le singole parole (lemmi) rimuovendo le parole inutili (stop words..)
def get_list_of_gains_words(sentence):
  list_words_lemma = []
  # aus_list_words = sentence.split()
  aus_list_words = word_tokenize(sentence)
  stop_words_list = get_stop_words()

  for w in aus_list_words:
    if w.lower() not in stop_words_list:
      pos_tag = get_wordnet_pos(w)
      if pos_tag != '':
        list_words_lemma.append(lemmatizer.lemmatize(w.lower(), pos_tag))

  return list_words_lemma


def read_text_file():
    return open("pasta2.1.txt", "r").read()

In [3]:
# il testo è la prima parte di: https://en.wikipedia.org/wiki/Pasta
# Wikipedia suddivide il testo così:
# 0 - 2
# 3 - 6
# 8 - 11
# 12 - 17
# 18 - 19

# restituisce un array in cui per ciascun elemento contiene le parole di una frase con associata la loro frequenza
# all'interno della frase
def get_sentences_with_tf(text):
  #sents_tf sara una lista che contiene tanti dizionari quante sono le frasi
  sents_tf = []
  #tokenized_sents contiene il nostro testo diviso per frasi, ('frase1', 'frase2'), sent_tokenize è un metodo di nltk
  tokenized_sents = sent_tokenize(text)
  for idx in range(len(tokenized_sents)):
    sents_tf.append({})
    #contiamo quante volte una determinata parola compare all interno della nostra frase "tokenized_sents"
    #usiamo FreqDist per calcolare la frequenza dei token
    fdist = FreqDist(word.lower() for word in word_tokenize(tokenized_sents[idx]))
    for word in word_tokenize(tokenized_sents[idx]):
      sents_tf[idx][word]=fdist.freq(word)
    
    #print("sents_tf " + str(sents_tf))

  return sents_tf


# funzione che segmenta il testo (text) in num_sections sezioni
def segment(text, num_sections):
  sents_tf = get_sentences_with_tf(text)
  #print(type(sents_tf))
  len_section = int(len(sents_tf) / (num_sections - 1)) # all'inizio sezioni stessa lunghezza
  #questa lista conterra gli indici dei vari break point, che all inizio saranno di uguale lunghezza e coincidono con
  #le varie sezioni 
  sections = [0]

  idx = len_section
  # Scorre le sezioni
  while idx <= len(sents_tf):

    #instanziamo min_sim a infinito e poi sara diminuito di volta in volta
    min_sim = math.inf
    idx_min_sim = 0
    # Scorre le frasi della sezione
    for jdx in range(idx - len_section, idx - 1): # cerchi nella sezione il punto di minimo
      #confrontiamo ogni frase con quella successiva, cercando la coppia di frasi che mi da il break point
      #ossia quelle piu dissimili tra di loro
      sim = similarity(sents_tf[jdx], sents_tf[jdx + 1])
      if sim < min_sim:
        min_sim = sim
        #mi salvo quali frasi mi davano il break point
        idx_min_sim = jdx

    # Dalla frase successiva al minimo, inizia l'altra sezione
    idx_min_sim += 1

    sections.append(idx_min_sim)
    #serve a passare alla sezione successiva, quindi passiaomo al blocco successivo(0,1,3,7..)
    idx += len_section

  sections.append(len(sents_tf))

  return sections


# è una misura di similarità che tiene conto delle frequenze delle parole per ogni frase.
# calcola l'overlap delle due frasi.
# Per ogni parola dell'overlap somma la media delle frequenze della parole all'interno delle due frasi
# Normalizza la somma sulla lunghezza dell'intersezione
def similarity(s1, s2):
  sim = 0
  overlap = set.intersection(set(s1.keys()), set(s2.keys()))

  if len(overlap) > 0:
    for word in overlap:
      sim += (s1[word] + s2[word]) / 2  # media fra le due frequenze

    #qui si normalizza sulla lunghezza delle intersezioni
    sim /= len(overlap)

  return sim


def print_results(sections,sentences):
  #per ogni sezione
  for idx in range(len(sections) - 1):
    #la sezione inizia alla riga  start_section e finsice alla riga end_section
    # alla posizione idx e idx+1 dell'array sections
    start_section = sections[idx]
    end_section = sections[idx + 1] #

    print("\n\n############################ Sezione", idx + 1, "############################")
    for jdx in range(start_section, end_section):
      print(sentences[jdx])


text = read_text_file()
num_sections = 5
sections = segment(text, num_sections)


#Stampo riga inizio e fine di ogni sezione (il secondo numero è escluso)
for idx in range(len(sections) - 1):
  print("Sezione", idx + 1, ":", sections[idx], "-", sections[idx + 1] - 1)




sentences = sent_tokenize(text)
# stampo il testo diviso per sezioni
print_results(sections,sentences)




Sezione 1 : 0 - 0
Sezione 2 : 1 - 5
Sezione 3 : 6 - 10
Sezione 4 : 11 - 12
Sezione 5 : 13 - 18


############################ Sezione 1 ############################
Pasta (US: /ˈpɑːstə/, UK: /ˈpæstə/; Italian pronunciation: [ˈpasta]) is a type of food typically made from an unleavened dough of wheat flour mixed with water or eggs, and formed into sheets or other shapes, then cooked by boiling or baking.


############################ Sezione 2 ############################
Rice flour, or legumes such as beans or lentils, are sometimes used in place of wheat flour to yield a different taste and texture, or as a gluten-free alternative.
Pasta is a staple food of Italian cuisine.
[1][2]
Pastas are divided into two broad categories: dried (pasta secca) and fresh (pasta fresca).
Most dried pasta is produced commercially via an extrusion process, although it can be produced at home.
Fresh pasta is traditionally produced by hand, sometimes with the aid of simple machines.


###################